# Trabajo Final de la materia Minería de Datos I

## Olimpia Saucedo Estrada y Francisco Ramírez Castañeda

### Preprocesamiento de Datos
Debido a que el conjunto de muestras contiene un mayor número de muestras maliciosas es necesario realizar un balanceo de estas mismas, por lo que se obtuvieron muestras maliciosas aleatoriamente del dataset ajustando el numero de muestras a 400 para posteriormente concatenar las muestras benignas. Con esto se obtuvo un conjunto de muestras balanceado.

Con el conjunto de muestras balanceado se realizó el preprocesamiento de los datos mediante los algoritmos TF-IDF y Word2Vec, los cuales permiten la extración de características para poder realizar el entrenamiento de los modelos de clasificación.

In [12]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from nltk.tokenize import word_tokenize
from sklearn import svm
from sklearn.pipeline import Pipeline
import features
from sklearn.linear_model import Perceptron
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier



In [2]:
import nltk
nltk.download('punkt')
  

[nltk_data] Downloading package punkt to /home/olibits/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dataset = pd.read_csv('dataset.csv')
df_minoria = dataset[dataset.y == 1]
df_mayoria = dataset[dataset.y == 0]
df_minoria_resample = resample(df_minoria, replace=True, n_samples=400, random_state=123)
dataset_balanceado = pd.concat([df_mayoria, df_minoria_resample])


In [4]:
#2 Caracterizar las muestras utilizando TF-IDF 
X,Y = dataset_balanceado['X'],dataset_balanceado['y']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
vec = TfidfVectorizer()


In [5]:
#2 Caracterizar las muestras utilizando Word2Vec
model_gn = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)
w2v = dict(zip(model_gn.index2word, model_gn.vectors))
X_tokens = [word_tokenize(s) for s in X]
X_trainw2v, X_testw2v, Y_trainw2v, Y_testw2v = train_test_split(X_tokens, Y, test_size=0.2, random_state=0)

In [6]:
perceptron = Perceptron()
perceptron_vec = Pipeline([('vectorizer', vec), ('pac', perceptron)])

perceptron_vec.fit(X_train, Y_train)
perceptron_score_vec = perceptron_vec.score(X_test, Y_test)
print(perceptron_score_vec)

perceptron_w2v = Pipeline([
    ("word2vec vectorizer", features.MeanEmbeddingVectorizer(w2v)),
    ("pac", perceptron)])
perceptron_vec.fit(X_train, Y_train)
perceptron_score_vec = perceptron_vec.score(X_test, Y_test)
print(perceptron_score_vec)

0.9361702127659575
0.9361702127659575


In [7]:
#TODO 3 Entrenar los algoritmos (Hay que dividirnos esta parte)
#maquina de soporte vectorial tfidf
svc = svm.SVC(C=10)
svm_clf = Pipeline([('vectorizer', vec), ('pac', svc)])
svm_clf.fit(X_train, Y_train)
svc_score = svm_clf.score(X_test, Y_test)
print(svc_score)

#maquina de soporte vectorial w2v
svm_w2v = Pipeline([
    ("word2vec vectorizer", features.MeanEmbeddingVectorizer(w2v)),
    ("pac", svc)])

svm_w2v.fit(X_trainw2v, Y_trainw2v)
svc_w2v_score=svm_w2v.score(X_testw2v, Y_testw2v)
print(svc_w2v_score)

0.8617021276595744
0.9042553191489362


In [11]:
#random forest
forest = RandomForestClassifier(n_estimators=100, max_depth = 100)

forest_sgdc = Pipeline([('vectorizer', vec), ('pac', forest)])
forest_sgdc.fit(X_train,Y_train)
forest_sgdc_score= forest_sgdc.score(X_test, Y_test)
print(forest_sgdc_score)

forest_sgdc_w2v = Pipeline([('vectorizer', features.MeanEmbeddingVectorizer(w2v)), ('pac', forest)])
forest_sgdc_w2v.fit(X_trainw2v, Y_trainw2v)
forest_sgdc_w2v_score=forest_sgdc_w2v.score(X_testw2v, Y_testw2v)
print(forest_sgdc_w2v_score)

0.9893617021276596
0.9680851063829787


In [15]:
knn = KNeighborsClassifier(n_neighbors = 3)

knn_vec = Pipeline([('vectorizer', vec), ('pac', knn)])
knn_vec.fit(X_train, Y_train)
knn_vec_score= knn_vec.score(X_test, Y_test)
print(knn_vec_score)

knn_w2v = Pipeline([('vectorizer', features.MeanEmbeddingVectorizer(w2v)), ('pac', knn)])
knn_w2v.fit(X_trainw2v, Y_trainw2v)
knn_w2v_score= knn_w2v.score(X_trainw2v, Y_trainw2v)
print(knn_w2v_score)

0.9468085106382979
0.9598930481283422


In [16]:
#Regresion Logistica, caracterizado con TF-IDF
lr = LogisticRegression(C=1000.0, random_state=0)
lr_clf = Pipeline([('vectorizer', vec), ('pac', lr)])
lr_clf.fit(X_train, Y_train)
lr_score = lr_clf.score(X_test,Y_test)
print(lr_score)

#Regresion Logistica, caracterizado con W2V
vecW2V = features.MeanEmbeddingVectorizer(w2v)
lr_w2v_clf = Pipeline([
    ("vectorizer", vecW2V),
    ("pac", lr)])

lr_w2v_clf.fit(X_trainw2v, Y_trainw2v)
lr_w2v_score=lr_w2v_clf.score(X_testw2v, Y_testw2v)
print(lr_w2v_score)


NameError: name 'LogisticRegression' is not defined

In [ ]:
#Árboles de decisión TF-IDF
dtc =  DecisionTreeClassifier(random_state=0)
dt_clf = Pipeline([('vectorizer', vec), ('pac', dtc)])
dt_clf.fit(X_train, Y_train)
dt_score = dt_clf.score(X_test, Y_test)
print(dt_score)
#Árboles de decisión w2v
dt_clf_w2v = Pipeline([('vectorizer', vecW2V), ('pac', dtc)])
dt_clf_w2v.fit(X_trainw2v, Y_trainw2v)
dt_clf_score = dt_clf_w2v.score(X_testw2v, Y_testw2v)
print(dt_clf_score)


In [ ]:
#TODO 4 Comparar los resultados

In [ ]:
#TODO 5 Graficar la matriz de confusión y curva ROC del mejor algoritmo

### Conclusiones